## Augment and Generate a Dataset Based on the 10 Objects

Use the 10 object types as the basis for creating a local dataset for fine-tuning the model
The data generated from this file was uploaded to Roboflow to create the final dataset used for fine-tuning

In [1]:
import os
import random
import cv2
from pathlib import Path
import albumentations as A

# Define the classes for the objects
classes = [
    'cell phone', 'remote', 'knife', 'book', 'spoon',
    'cup', 'scissors', 'fork', 'toothbrush', 'ball'
]

# Paths for the dataset
images_dir = "../Objects/"
output_dir = "local_dataset/"

# Create dataset structure
os.makedirs(output_dir, exist_ok=True)
for subset in ['train', 'val', 'test']:
    os.makedirs(f"{output_dir}/images/{subset}", exist_ok=True)
    os.makedirs(f"{output_dir}/labels/{subset}", exist_ok=True)

# Data augmentation pipeline using Albumentations
augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=25, p=0.5),
    A.RandomScale(scale_limit=0.2, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5)
])

# Dataset split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

def get_class_id(image_path):
    """
    Get the class ID for an image based on its file path.
    Classify the 10 objects into their correct class.

    Args:
        image_path (str): The path to the image file.

    Returns:
        int: The class ID corresponding to the image.
    """
    class_ids = {
        "../Objects/O1.jpg": 1,
        "../Objects/O2.jpg": 2,
        "../Objects/O3.jpg": 3,
        "../Objects/O4.jpg": 4,
        "../Objects/O5.jpg": 5,
        "../Objects/O6.jpg": 6,
        "../Objects/O7.jpg": 7,
        "../Objects/O8.jpg": 8,
        "../Objects/O9.jpg": 9,
        "../Objects/O10.jpg": 10
    }

    return class_ids.get(image_path)

def augment_and_save(image_path, class_id, output_dir, subset, num_augmentations=10):
    """
    Augment and save images with their labels.

    Args:
        image_path (Path): Path to the input image.
        class_id (int): Class ID of the image.
        output_dir (str): Base directory to save the augmented images and labels.
        subset (str): Subset (train, val, or test) to which the image belongs.
        num_augmentations (int): Number of augmented images to generate. Default is 10.
    """
    image = cv2.imread(str(image_path))
    if image is None:
        raise ValueError(f"Failed to read image at {image_path}")

    bbox = [0.2, 0.2, 0.8, 0.8]  # Bounding box with normalized x_center, y_center, width, height
    annotations = f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n"

    for i in range(num_augmentations):
        # Apply the augmentations
        augmented = augmentations(image=image)
        augmented_image = augmented["image"]
        augmented_image_name = f"{image_path.stem}_{subset}_{i}.jpg"
        cv2.imwrite(f"{output_dir}/images/{subset}/{augmented_image_name}", augmented_image)

        # Save corresponding label
        label_path = f"{output_dir}/labels/{subset}/{augmented_image_name.replace('.jpg', '.txt')}"
        with open(label_path, 'w') as f:
            f.write(annotations)

# Load and shuffle all images (to avoid all of a certain image being categorized in one set)
all_images = list(Path(images_dir).glob("*.jpg"))
random.shuffle(all_images)

# Split images into train, val, and test subsets
num_images = len(all_images)
train_images = all_images[:int(num_images * train_ratio)]
val_images = all_images[
    int(num_images * train_ratio):int(num_images * (train_ratio + val_ratio))
]
test_images = all_images[int(num_images * (train_ratio + val_ratio)):]

# Augment and save images for each subset
dataset_combined = zip(
    ['train', 'val', 'test'], 
    [train_images, val_images, test_images]
)

for subset, subset_images in dataset_combined:
    for image_path in subset_images:
        class_id = get_class_id(str(image_path))
        if class_id is None:
            print(f"Skipping unrecognized image: {image_path}")
            continue
        augment_and_save(image_path, class_id, output_dir, subset)

# Create data.yaml for exporting to Roboflow 
# which will ultimately be used to train a YOLO model
yaml_content = f"""
path: {output_dir}
train: images/train
val: images/val
test: images/test

names:
"""
for i, cls in enumerate(classes):
    yaml_content += f"  {i}: {cls}\n"

with open(f"{output_dir}/data.yaml", 'w') as f:
    f.write(yaml_content)

print("Dataset with augmented images created successfully!")

Dataset with augmented images created successfully!
